<a href="https://colab.research.google.com/github/GayathriShrikanth/J046NLP/blob/master/CountVec_J046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df=pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [2]:
import pandas as pd
df_twitter=pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")
df_twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from string import digits

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
def corpus(text):
  text=text.lower()
  remove_digits = str.maketrans('', '', digits)
  text = text.translate(remove_digits)
  text = re.sub(r'[^\w\s]',' ',text)
  return text

In [0]:
final=df_twitter.TweetText.apply(corpus)

In [6]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase=True, stop_words=set(stopwords.words('english')))
X = vectorizer.fit_transform(final[:50])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
print(vectorizer.get_feature_names())

['actually', 'adding', 'affair', 'ainteeentrippin', 'air', 'amazing', 'amount', 'android', 'announced', 'apologize', 'app', 'apple', 'apply', 'appreciated', 'apps', 'appstore', 'argument', 'aside', 'awesome', 'azeev', 'backchannel', 'bahhh', 'battery', 'best', 'better', 'bit', 'bitching', 'bkad', 'blackberry', 'blessed', 'body', 'bound', 'brand', 'break', 'brilliance', 'bring', 'bro', 'brother', 'burn', 'bus', 'buy', 'call', 'carrier', 'cc', 'ceo', 'ceremoniously', 'champ', 'close', 'co', 'coach', 'come', 'companies', 'corporate', 'covent', 'crack', 'cracked', 'current', 'currently', 'customer', 'dad', 'dailydealchat', 'danielpink', 'dark', 'days', 'deals', 'desktop', 'developer', 'devices', 'dis', 'disappointed', 'done', 'downloads', 'drain', 'duet', 'dvjlsi', 'dying', 'easy', 'education', 'efffing', 'enough', 'event', 'ever', 'everyone', 'evo', 'exbnqjy', 'experience', 'far', 'feature', 'featured', 'find', 'first', 'fixed', 'flashcard', 'fmjxtmm', 'free', 'gamerchik', 'garden', 'gave

In [8]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [0]:
features=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [10]:
features.head()

,actually,adding,affair,ainteeentrippin,air,amazing,amount,android,announced,apologize,app,apple,apply,appreciated,apps,appstore,argument,aside,awesome,azeev,backchannel,bahhh,battery,best,better,bit,bitching,bkad,blackberry,blessed,body,bound,brand,break,brilliance,bring,bro,brother,burn,bus,...,twaeudbp,twitter,uber,ukwokbgd,umber,unexpected,units,unwanted,upcoming,useful,useless,user,users,uxawkwf,version,via,video,waiting,want,wants,warranty,way,website,weekend,well,whores,wiped,wish,working,workshop,wow,wristpad,ya,years,yesterday,yourdaysarenumbered,youtube,ywvbez,zappos,你好
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
y=df_twitter["Sentiment"][:50]

In [12]:
features.shape

(50, 305)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts,GridSearchCV
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test = tts(features, y,test_size=0.3, random_state=30)

In [0]:
model = RandomForestClassifier()

In [0]:
params = {'max_depth': [10, 20, 30, None],
 'max_features': ['sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': range(10,50,10)}
 
gs = GridSearchCV(estimator = model, param_grid = params, 
                          cv = 5, n_jobs = 10, verbose = 1)

In [25]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.3s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   16.8s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   27.6s
[Parallel(n_jobs=10)]: Done 720 out of 720 | elapsed:   40.4s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [26]:
print(classification_report(gs.predict(X_test),y_test))

              precision    recall  f1-score   support

    positive       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

